In [1]:
from math import floor, fabs
from time import sleep
from math import fabs, isnan
import os
from glob import glob
from lammps import lammps, PyLammps
from time import sleep
from shutil import copyfile
import natsort
import ase.io
from ase import Atoms
from ase.build import fcc111, add_adsorbate, bulk
from ase.io import read, write,vasp
from glob import glob

import sys

In [2]:
def GetXSFFrame(infile):
    last_pos = infile.tell()
    line = infile.readline()
    if line == '':
        raise IOError
    col = line.split()

    for item in col:
        try:
#            print item
            energy = float(item)
            break
        except:
            continue
    else:
        raise FormatError
    header = []
    header.append(line)


    coords = []
 
    blankCnt = 0
    while True: 
        last_pos = infile.tell()
        line = infile.readline()
        if line == '':
            blankCnt += 1
            if blankCnt > 3:
#                raise IOError
                return header, coords, energy
#        print line
        col = line.split()
        if len(col) < 4:
            header.append(line)
        elif col[0] == "Cu":
            coords.append(line)
        elif col[0] == "#":
            infile.seek(last_pos)
            break
        
    return header, coords, energy

In [ ]:
def runsim2(lmp, PyL, struct):
    fileid = struct.replace(".data","")
    lmp.command("clear")
    lmp.command("boundary p p p")
    lmp.command("box tilt large")
    lmp.command("units metal")
    lmp.command("read_data "+struct)
    lmp.command("replicate 3 3 1")
    lmp.command("pair_style eam")
    lmp.command("pair_coeff  * * ../Cu_SR2.eam")
    lmp.command("neigh_modify delay 0 every 1 check no")
    lmp.command("neigh_modify one 10000")
    lmp.command("mass 1 63.0")
    lmp.command("fix 1 all box/relax x 0.0 y 0.0 vmax 0.001")
    lmp.command("minimize 1.0e-6 1.0e-2 2000 2000")
    lmp.command("run 0 pre no")
    lmp.command('variable potential equal pe')
    value = lmp.extract_variable('potential',None, 0)

    return value

In [ ]:
from ase.io import read
import numpy as np

lmp = lammps()
PyL = PyLammps(ptr=lmp)

FCC_struct = "./Cu_FCC.dat"
eTarg_FCC = runsim2(lmp, PyL,FCC_struct)
eTarg_FCC = -eTarg_FCC/(256*9) 
structlist1 = glob("./*.data")
nnvsref1 = open("Surf.dat", "w")
E_FCC = (960.09464893/256) - 0.1909186
Unit_c = (6.242*10**(-5)
for struct in structlist1:
    miller = struct.split("(")[-1].split(")")[0]
    # Step 1: Read the LAMMPS data file
    structure = read(struct.replace(".data",".xsf"), format="xsf")
    with open(struct.replace(".data",".xsf"), "r") as infile:
        header, coords, energy = GetXSFFrame(infile)
    
    atomcount = len(structure.get_positions())
    print(energy)
    energy = energy + 0.1909186*atomcount
    # Step 2: Extract the lattice vectors
    cell = structure.cell
    a1 = cell[0]  # Lattice vector along x
    a2 = cell[1]  # Lattice vector along y
    # Step 3: Calculate the xy-area
    xy_area = np.linalg.norm(np.cross(a1, a2))
    eTarg = runsim2(lmp, PyL,struct)
    print(eTarg)
    surfref = ( energy +(E_FCC*atomcount))/(Unit_c*xy_area*2)
    surf = (eTarg+(eTarg_FCC*atomcount*9))/(Unit_c*xy_area*2*9)
    nnvsref1.write("%s %s %s\n"%(miller , surf,surfref))
nnvsref1.close()